In [ ]:
import numpy as np
import h5py
import os
import pydicom
import pandas as pd
import math
from sklearn.preprocessing import normalize
import matplotlib.pyplot as plt
from itertools import compress
from scipy import ndimage

## Config variables

In [ ]:
PATH_DICOM = '../MRIs/DICOM'
PATH_H5 = '../data/testdata_dicom.h5'
PATH_BASELINE = '../data/baseline_data_DWI.csv'
IMAGE_DIMENSIONS = (128, 128, 100)

## Functions

In [ ]:
def get_fov(ref_file, image_dims_3d):
    ConstPixelSpacing = (float(ref_file.PixelSpacing[0]), float(ref_file.PixelSpacing[1]), float(ref_file.SliceThickness))

    x_fov = (image_dims_3d[0]+1)*ConstPixelSpacing[0]
    y_fov = (image_dims_3d[1]+1)*ConstPixelSpacing[1]
    z_fov = (image_dims_3d[2]+1)*ConstPixelSpacing[2]

    return x_fov, y_fov, z_fov

def get_pixel_spacing(image_fov, image_dims_3d):

    x = np.arange(0.0, image_fov[0], image_fov[0] / image_dims_3d[0])
    y = np.arange(0.0, image_fov[1], image_fov[1] / image_dims_3d[1])
    z = np.arange(0.0, image_fov[2], image_fov[2] / image_dims_3d[2])

    return x, y, z

def load_images(file_list):
    
    ## loads DICOM images from file_list and creates a 3D array with pixel data
    
    ref_file = pydicom.read_file(file_list[0])
    image_dims_3d = (int(ref_file.Rows), int(ref_file.Columns), len(file_list))
    # print("Original image size: {}".format(image_dims_3d))

    image_fov = get_fov(ref_file, image_dims_3d)
    original_spacing = get_pixel_spacing(image_fov, image_dims_3d)
    origArray = np.zeros(image_dims_3d, dtype=ref_file.pixel_array.dtype)
    origArray.shape

    # loop through all the DICOM files
    removed = 0
    file_list.sort()
    for filename in file_list:
        # read the file
        ds = pydicom.read_file(filename)
        # store the raw image data

        if(ds.Rows != ref_file.Rows):
            removed  += 1
            continue

        origArray[:, :, file_list.index(filename)] = ds.pixel_array

    if(removed != 0):
        print('removed {} files for sequence {}'.format(removed, ref_file.SequenceName))

    return origArray #, image_fov, original_spacing

def normalize_array(array):
    min = np.min(array)
    max = np.max(array)
    normalized = (array - min) / (max - min)
    return normalized

def scale_array(array, dims):
    
    ## scales array to IMAGE_DIMENSIONS and normalizes to [0,1]
    
    scaling_factor = [dims[0]/array.shape[0], dims[1]/array.shape[1], dims[2]/array.shape[2]]
    scaledArray = ndimage.zoom(array, scaling_factor)
    scaledArray = normalize_array(scaledArray)

    # print("Scaled image size: {}".format(dims))
    return scaledArray

def plot_all_slices(data, orientation = 'axial', vmax = False):
    
    pixel_spacing = np.arange(0.0, IMAGE_DIMENSIONS[0], 1), \
                    np.arange(0.0, IMAGE_DIMENSIONS[1], 1), \
                    np.arange(0.0, IMAGE_DIMENSIONS[2], 1), 
    
    if orientation == 'axial':
        n_slices = min(data.shape[2], 32) # plot maximum 32 slices
    else:
        n_slices = 32
    n_cols = 8
    n_rows = math.ceil(n_slices / n_cols)
    aspect_ratio = 1
    base_size = 2
    fig_size = (n_cols*base_size/aspect_ratio, n_rows*base_size)
    fig = plt.figure(figsize=fig_size)

    if orientation == 'axial':
        indeces = np.linspace(start=0, stop=data.shape[2]-1, num=n_slices)
    else:
        indeces = np.linspace(start=0, stop=data.shape[0]-1, num=n_slices)

    sp = 1
    for index in np.nditer(indeces.astype(int)):
        if orientation == 'coronal':
            image = np.rot90(data[index, :, ], k = 3)
            a = pixel_spacing[0]
            b = pixel_spacing[2]
        elif orientation == 'sagital':
            image = np.rot90(data[:, index, :], k = 3)
            a = pixel_spacing[1]
            b = pixel_spacing[2]
        else:
            image = np.rot90(data[:, :, index], k = 2)
            a = pixel_spacing[0]
            b = pixel_spacing[1]
        ax = fig.add_subplot(n_rows, n_cols, sp)
        if vmax:
            ax.pcolormesh(a, b, image, cmap="gray", vmin = 0, vmax = vmax)
        else:
            ax.pcolormesh(a, b, image, cmap="gray")
        ax.set_aspect('equal', 'box')
        ax.set_axis_off()
        sp += 1

    fig.tight_layout(pad=0.0)
    
    plt.savefig("../data/*re_b1000t.jpg", dpi=300)

## Import Baseline Dataset

In [ ]:
baseline_data = pd.read_csv(PATH_BASELINE, sep = " ")
baseline_data.p_id = [format(id, '03d') for id in baseline_data.p_id] 
baseline_data.p_id

## Load all paths to images

In [ ]:
X = []
listFilesDICOM = []

for paths, dirnames, filenames in os.walk(PATH_DICOM, topdown = True):
    for file in filenames:
        if file != 'DICOMDIR' and file[-2:] != 'h5':
            listFilesDICOM.append(os.path.join(paths,file))

data_info = pd.DataFrame()
data_info['patient'] = pd.Series()
data_info['sequence'] = pd.Series()
data_info['description'] = pd.Series()thundthun
data_info['aquisition_time'] = pd.Series()
data_info['rows'] = pd.Series()
data_info['columns'] = pd.Series()
data_info['filepath'] = pd.Series()

for index, file in enumerate(listFilesDICOM):
    DICOM_file = pydicom.read_file(file)
    data_info.loc[index, 'patient'] = file[14:17]
    data_info.loc[index, 'sequence'] = DICOM_file.SequenceName ## 0018 0024
    data_info.loc[index, 'description'] = DICOM_file.SeriesDescription ## 0008 103e
    data_info.loc[index, 'series_number'] = DICOM_file.SeriesNumber ## 0020 0011
    data_info.loc[index, 'aquisition_time'] = DICOM_file.AcquisitionTime ## 0008 0032
    data_info.loc[index, 'instance_uid'] = DICOM_file.SOPInstanceUID   ## 0008 0018 --> one per image
    data_info.loc[index, 'reference_study'] = DICOM_file.ReferencedStudySequence[0].ReferencedSOPInstanceUID  ## 0008 1110 --> one per patient
    # data_info.loc[index, 'creation_time'] = DICOM_file.InstanceCreationTime ## 0008 0013 --> not always present
    data_info.loc[index, 'rows'] = DICOM_file.Rows ## 0028 0010
    data_info.loc[index, 'columns'] = DICOM_file.Columns ## 0028 0011
    data_info.loc[index, 'filepath'] = file
    # TODO: add pixel spacing

## DICOM header

In [ ]:
listFilesDICOM[0:10]

In [ ]:
# ref_file = pydicom.read_file(listFilesDICOM[0])
# ref_file
# ref_file.ReferencedStudySequence[0].ReferencedSOPInstanceUID  

## Import DICOM images to h5

In [ ]:
baseline_data.p_id

In [ ]:
first_patient = True
os.remove(PATH_H5)
with h5py.File(PATH_H5, 'a') as f:
    for patient_number in set(data_info.patient):
        
        print('loading sequence for patient {}'.format(patient_number))
        if patient_number not in list(baseline_data.p_id):
            print('patient {} has no baseline values. Sequence will not be loaded. \n'.format(patient_number))
            continue

        IMAGE_NAME = []
        X = []
        
        ## load baseline values
        PATIENT_NB =  [patient_number]
        Y = baseline_data.stroke.loc[baseline_data.p_id == patient_number].values
        
        ## copy patient specific filepaths
        patient_data = data_info[data_info.patient == patient_number].copy()
        
        ## check if patient has a b1000t sequence
        in_set = []
        for i in set(patient_data.sequence):
            in_set.append('b1000t' in i)
        
        ## if patient has no or more than one b1000t sequence: choose sequence by hand
        if sum(in_set) != 1:
            ## --> load images in amide, choose image by name
            print('which image should be loaded?')
            for index, image in enumerate(set(patient_data.description)):
                print('{} - {}'.format(index, image))
            choice = input('>')
            chosen_sequence = list(set(patient_data.description))[int(choice)]
            
            print('{} {} \n'.format(' '.ljust(15), chosen_sequence))
            file_list = list(patient_data.filepath[patient_data.description == chosen_sequence])
            raw_3d_image = load_images(file_list)
            scaled_3d_image = scale_array(raw_3d_image, IMAGE_DIMENSIONS)


            IMAGE_NAME.append(chosen_sequence)
            X.append(scaled_3d_image)
            
        ## patient has a b1000t sequence: choose sequence automatically
        else:
        
            for sequence_type in set(patient_data.sequence):
            
                if 'b1000t' in sequence_type:
                
                    print('{} {} \n'.format(sequence_type.ljust(15), \
                                    set(patient_data.description[patient_data.sequence == sequence_type])))
                    file_list = list(patient_data.filepath[patient_data.sequence == sequence_type])
                    raw_3d_image = load_images(file_list)
                    scaled_3d_image = scale_array(raw_3d_image, IMAGE_DIMENSIONS)

#                     PATIENT_NB.append(patient_number)
                    IMAGE_NAME.append(sequence_type)
                    X.append(scaled_3d_image)
            
                else:
                    continue
                
        PATIENT_NB = np.string_(PATIENT_NB)
        IMAGE_NAME = np.string_(IMAGE_NAME)
        X = np.array(X)
        Y = np.array(Y)
        
        ## write to h5py sequentially
        if first_patient: ## initialize dataset
            f.create_dataset('X', data = X, maxshape = (None, 128, 128, 128), chunks = True)
            f.create_dataset('Y', data = Y, maxshape = (None,), chunks = True)
            f.create_dataset('PATIENT_NB', data = PATIENT_NB, maxshape = (None,), chunks = True)
            f.create_dataset('IMAGE_NAME', data = IMAGE_NAME, maxshape = (None,), chunks = True)
            first_patient = False
            
        else: ## append dataset
            f['X'].resize((f['X'].shape[0] + X.shape[0]), axis = 0)
            f['X'][-X.shape[0]:, :, :, :] = X
            f['Y'].resize((f['Y'].shape[0] + Y.shape[0]), axis = 0)
            f['Y'][-Y.shape[0]:] = Y
            f['PATIENT_NB'].resize((f['PATIENT_NB'].shape[0] + PATIENT_NB.shape[0]), axis = 0)
            f['PATIENT_NB'][-PATIENT_NB.shape[0]:] = PATIENT_NB
            f['IMAGE_NAME'].resize((f['IMAGE_NAME'].shape[0] + IMAGE_NAME.shape[0]), axis = 0)
            f['IMAGE_NAME'][-IMAGE_NAME.shape[0]:] = IMAGE_NAME


## Sequence Types

In [ ]:
set(data_info['sequence'])

In [ ]:
set(data_info['description'])

- Some Patients have the same description for two differnet sequences
- Sequence Types are unique (every sequence has a different name)
- But: Patient 4 lacks all Sequence Types
- Patient 8 has different names for sequence types

## Plots

In [ ]:
dd = h5py.File(PATH_H5, 'r')
patients = [p.decode() for p in dd['PATIENT_NB']]
patients

In [ ]:
def load_from_h5(patient_nb):
    dd = h5py.File(PATH_H5, 'r')
    patients = [p.decode() for p in dd['PATIENT_NB']]

    mri = np.array(dd['X'])[patients.index(patient_nb), :, :, :]
    return mri.reshape(IMAGE_DIMENSIONS)

In [ ]:
mri_sequence = load_from_h5(patient_nb = '001')
plot_all_slices(mri_sequence, 'axial', vmax = 1)

In [ ]:
mri_sequence = load_from_h5(patient_nb = '06')
plot_all_slices(mri_sequence, 'axial', vmax = 1)

In [ ]:
mri_sequence = load_from_h5(patient_nb = '04')
plot_all_slices(mri_sequence, 'axial', vmax = 1)